<a href="https://colab.research.google.com/github/nawnie/EveryDream2trainer/blob/main/Train_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <img src="https://github.com/victorchall/EveryDream2trainer/blob/562c4341137d1d9f5bf525e6c56fb4b1eefa2b57/doc/ed_logo_comp.jpg?raw=true" width="600" height="300">
</p>

<br>

---

<div align="center">
  <font size="6" color="yellow">Colab Edition</font>
</div>

---

<br>

Check out the **EveryDream2trainer** documentation and runpod/vastai and local setups here: 

[📚 **Documentation**](https://github.com/victorchall/EveryDream2trainer#docs)

And join our vibrant community on Discord:

[💬 **Join the Discord**](https://discord.gg/uheqxU6sXN)

If you find this tool useful, please consider subscribing to the project on Patreon or making a one-time donation on Ko-fi. Your donations keep this project alive as a free open-source tool with ongoing enhancements.

<br>

<p align="center">
  <a href="https://www.patreon.com/everydream">
    <img src="https://github.com/victorchall/EveryDream2trainer/raw/main/.github/patreon-medium-button.png?raw=true" width="200" height="50">
  </a>
</p>

<br>

<p align="center">
  <a href="https://ko-fi.com/everydream">
    <img src="https://github.com/victorchall/EveryDream2trainer/raw/main/.github/kofibutton_sm.png?raw=true" width="75" height="75">
  </a>
</p>


In [ ]:
#@markdown # Setup and Install Dependencies
from IPython.display import clear_output, display, HTML
import subprocess
import time
import os 
from tqdm.auto import tqdm
import PIL

# Defining function for colored text
def colored(r, g, b, text):
    return f"\033[38;2;{r};{g};{b}m{text} \033[38;2;255;255;255m"

#@markdown Optional connect Gdrive  But strongly recommended
#@markdown This will let you put all your training data and checkpoints directly on your drive.  
#@markdown Much faster/easier to continue later, less setup time.

#@markdown Creates /content/drive/MyDrive/everydreamlogs/ckpt
Mount_to_Gdrive = True #@param{type:"boolean"} 

# Clone the git repository
print(colored(0, 255, 0, 'Cloning git repository...'))
!git clone https://github.com/victorchall/EveryDream2trainer.git

if Mount_to_Gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

  !mkdir -p /content/drive/MyDrive/everydreamlogs/ckpt

%cd /content/EveryDream2trainer

# Download Arial font
print(colored(0, 255, 0, 'Downloading Arial font...'))
!wget -O arial.ttf https://raw.githubusercontent.com/matomo-org/travis-scripts/master/fonts/Arial.ttf

packages = [
    'transformers==4.29.2',
    'diffusers[torch]==0.17.1',
    'pynvml==11.4.1',
    'bitsandbytes==0.41.1',
    'ftfy==6.1.1',
    'aiohttp==3.8.4',
    'compel~=1.1.3',
    'protobuf==3.20.1',
    'wandb==0.15.3',
    'pyre-extensions==0.0.29',
    'xformers==0.0.20',
    'pytorch-lightning==1.6.5',
    'OmegaConf==2.2.3',
    'tensorboard>=2.11.0',
    'tensorrt'
    'wandb',
    'colorama',
    'keyboard',
    'lion-pytorch',
    'safetensors',
    'torchsde'
]

print(colored(0, 255, 0, 'Installing packages...'))
for package in tqdm(packages, desc='Installing packages', unit='package', bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}'):
    if isinstance(package, tuple):
        package_name, extra_index_url = package
        cmd = f"pip install -I -q {package_name} --extra-index-url {extra_index_url}"
    else:
        cmd = f"pip install -q {package}"
        
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

clear_output()


# Execute Python script
print(colored(0, 255, 0, 'Executing Python script...'))
!python utils/get_yamls.py
clear_output()

print(colored(0, 255, 0, "DONE! installing dependencies."))

# Import pynvml and get GPU details
import pynvml

pynvml.nvmlInit()

handle = pynvml.nvmlDeviceGetHandleByIndex(0)

gpu_name = pynvml.nvmlDeviceGetName(handle)
gpu_memory = pynvml.nvmlDeviceGetMemoryInfo(handle).total / 1024**3
cuda_version_number = pynvml.nvmlSystemGetCudaDriverVersion_v2()
cuda_version_major = cuda_version_number // 1000
cuda_version_minor = (cuda_version_number % 1000) // 10
cuda_version = f"{cuda_version_major}.{cuda_version_minor}"

pynvml.nvmlShutdown()

Python_version = !python --version
import torch
import torchvision
import xformers

display(HTML(f"""
<table style="background-color:transparent;">
  <tr>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">Python version:</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">{Python_version[0]}</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">GPU Name:</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">{gpu_name}</span></td>
  </tr>
  <tr>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">PyTorch version:</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">{torch.__version__}</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">GPU Memory (GB):</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">{gpu_memory:.2f}</span></td>
  </tr>
  <tr>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">Torchvision version:</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">{torchvision.__version__}</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">CUDA version:</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">{cuda_version}</span></td>
  </tr>
  <tr>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">XFormers version:</span></td>
    <td style="background-color:transparent;"><span style="color: #FFFF00;">{xformers.__version__}</span></td>
  </tr>
</table>
"""))

time.sleep(2)

In [ ]:
#@title Get A Base Model
#@markdown Choose SD1.5, Waifu Diffusion 1.3, SD2.1, or 2.1(512) from the dropdown, or paste your own URL in the box
#@markdown * Alternately you can link to an HF repo using NAME/MODEL, this does not save to your Gdrive, if you want to save an HF model, use the direct URL

#@markdown * Link to a set of diffusers on your Gdrive

#@markdown * **SD2.1 models require HIGH RAM SHAPE** (system ram) as the 12.7GB in free tier isn't big enough.  T4 (16GB) GPU is enough for both SD1.5 and SD2.1 but you may need to adjust batch size down for SD2.1.

#@markdown * Paste a URL, atm there is no support for .safetensors

from IPython.display import clear_output
!mkdir input
%cd /content/EveryDream2trainer
MODEL_LOCATION = "panopstor/EveryDream" #@param ["sd_v1-5+vae.ckpt", "hakurei/waifu-diffusion-v1-3", "stabilityai/stable-diffusion-2-1-base", "stabilityai/stable-diffusion-2-1"] {allow-input: true}

if MODEL_LOCATION == "sd_v1-5+vae.ckpt":
  MODEL_LOCATION = "panopstor/EveryDream"

If_Ckpt = False
import os

download_path = ""

if ".co" in MODEL_LOCATION or "https" in MODEL_LOCATION or "www" in MODEL_LOCATION:
    MODEL_URL = MODEL_LOCATION
    print("Downloading...")
    !wget $MODEL_LOCATION
    clear_output()
    print("Download completed!")
    download_path = os.path.join(os.getcwd(), os.path.basename(MODEL_URL))
else:
    save_name = MODEL_LOCATION

%cd /content/EveryDream2trainer

inference_yaml = " "

# Check if the downloaded or copied model is a .ckpt file
#@markdown Is the model 1.5 or 2.1 based?
model_type = "SD1x" #@param ["SD1x", "SD2_512_base", "SD21"]

if download_path.endswith(".ckpt") or MODEL_LOCATION.endswith(".ckpt"):
    If_Ckpt = True
    save_path = download_path
    if ".ckpt" in save_name:
        save_name = save_name.replace(".ckpt", "")
    
    img_size = 512
    upscale_attention = False
    prediction_type = "epsilon"

    if model_type == "SD1x":
        inference_yaml = "v1-inference.yaml"
    elif model_type == "SD2_512_base":
        upscale_attention = True
        inference_yaml = "v2-inference.yaml"
    elif model_type == "SD21":
        upscale_attention = True
        prediction_type = "v_prediction"
        inference_yaml = "v2-inference-v.yaml"
        img_size = 768

    !python utils/convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim \
    --original_config_file $inference_yaml \
    --image_size $img_size \
    --checkpoint_path $MODEL_LOCATION \
    --prediction_type $prediction_type \
    --upcast_attn False \
    --dump_path $save_name

# Set the save path to the GDrive directory if cache_to_gdrive is True
if If_Ckpt:
    save_name = os.path.join("/content/drive/MyDrive/everydreamlogs/ckpt", save_name)

if inference_yaml != " ":
    print("Model saved to: " + save_name + ". The " + inference_yaml + " was used!")

print("Model " + save_name + " will be used!")

# Training

For a more indepth Explanation of each of these paramaters check out /content/EveryDream2trainer/doc.

After youve tried a few models you will find /content/EveryDream2trainer/doc/ADVANCED_TWEAKING.md to be extremly helpful.

In [ ]:
from google.colab import runtime
from IPython.display import clear_output, display
import time
import sys
import time
import shutil
import json
import os

#@title  #Run Everydream 2
%cd /content/EveryDream2trainer
#@markdown If you want to use a .json config or upload your own, skip this cell and run the cell below instead
!rm -r /content/EveryDream2trainer/Training_Data
#@markdown * Save logs and output ckpts to Gdrive (strongly suggested, will need space)
Save_to_Gdrive = True #@param{type:"boolean"}
#@markdown * Save extra state information to help continue training (if you will resume later, requires high RAM shape)
Save_optimizer_state = False #@param{type:"boolean"}
#@markdown * Disconnect after training to save Credits 
Disconnect_after_training =  False #@param{type:"boolean"}
#@markdown * Use resume to contnue training you just ran, will also find latest diffusers log in your Gdrive to continue.
resume = False #@param{type:"boolean"}
#@markdown * Name your project so you can find it in your logs
Project_Name = "My_Project" #@param{type: 'string'}



if model_type == 'SD2_512_base' or model_type == 'SD21':
    file_path = "/content/EveryDream2trainer/optimizerSD21.json"
else:
    file_path = "/content/EveryDream2trainer/optimizer.json"

with open(file_path, 'r') as file:
    data = json.load(file)


#@markdown * The learning rate affects how much "training" is done on the model per training step. It is a very careful balance to select a value that will learn your data and not wreck the model. 
#@markdown Leave this default unless you are very comfortable with training and know what you are doing.
Learning_Rate = 1e-6 #@param{type: 'number'}
#@markdown * chosing this will allow you to ignore any settings specific to the text encode and will match it with the Unets settings, recommended for beginers.
Match_text_to_Unet = False #@param{type:"boolean"}
Text_lr =  5e-7 #@param {type:"number"}
#@markdown * A learning rate scheduler can change your learning rate as training progresses.
#@markdown * I recommend sticking with constant until you are comfortable with general training. 
Schedule = "constant" #@param ["constant", "polynomial", "linear", "cosine"] {allow-input: true}
Text_lr_scheduler = "cosine" #@param ["constant", "polynomial", "linear", "cosine"] {allow-input: true}
#@markdown * warm up steps are useful for validation and cosine lrs
lr_warmup_steps = 0 #@param{type:"integer"}
lr_decay_steps = 0 #@param {type:"number"}         
Text_lr_warmup_steps = 0 #@param {type:"number"}
Text_lr_decay_steps = 0 #@param {type:"number"}         

if Match_text_to_Unet:
  Text_lr = Learning_Rate
  Text_lr_scheduler = Schedule
  Text_lr_warmup_steps = lr_warmup_steps

data['base']['lr'] = Learning_Rate
data['text_encoder_overrides']['lr'] = Text_lr 
data['base']['lr_scheduler'] = Schedule
data['text_encoder_overrides']['lr_scheduler'] = Text_lr_scheduler
data['base']['lr_warmup_steps'] = lr_warmup_steps
data['base']['lr_decay_steps'] = lr_decay_steps
data['text_encoder_overrides']['lr_warmup_steps'] = Text_lr_warmup_steps
data['text_encoder_overrides']['lr_decay_steps'] = Text_lr_decay_steps

# Save the updated JSON data back to the file
with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)

#@markdown * Resolution to train at (recommend 512).  Higher resolution will require lower batch size (below).
Resolution = 512 #@param {type:"slider", min:256, max:768, step:64}

#@markdown * Batch size is also another "hyperparameter" of itself and there are tradeoffs. It may not always be best to use the highest batch size possible. Once of the primary reasons to change it is if you get "CUDA out of memory" errors where lowering the value may help.

#@markdown * Batch size impacts VRAM use.  8 should work on SD1.x models and 5 for SD2.x models at 512 resolution.  Lower this if you get CUDA out of memory errors. You can check resources on your instance and watch the GPU RAM.

Batch_Size = 6 #@param{type: 'number'}

#@markdown * Gradient accumulation is sort of like a virtual batch size increase use this to increase batch size with out increasing vram usage
#@markdown Increasing from 1 to 2 will have a minor impact on vram use, but more beyond that will not.
#@markdown In colab free teir you can expect the fastest proformance from a batch of 8 and a gradient step of 1
#@markdown This is mostly for use if you are training higher resolution on free tier and cannot increase batch size.

Gradient_steps = 1 #@param{type:"slider", min:1, max:10, step:1}

#@markdown * Location on your Gdrive where your training images are.
Dataset_Location = "/content/drive/MyDrive/training_samples" #@param {type:"string"}

if not resume:
  model = save_name

#@markdown * Max Epochs to train for, this defines how many total times all your training data is used. Default of 100 is a good start if you are training ~30-40 images of one subject. If you have 100 images, you can reduce this to 40-50 and so forth.

Max_Epochs = 100 #@param {type:"slider", min:0, max:200, step:1}

#@markdown * How often to save checkpoints.
Save_every_N_epoch = 20 #@param{type:"integer"}

#@markdown * Test sample generation steps, how often to generate samples during training.

#@markdown You can set your own sample prompts by adding them, one line at a time, to `/content/EveryDream2trainer/sample_prompts.txt`.  If left empty, it will use the captions from your training images.

#@markdown Use the steps_between_samples to set how often the samples are generated.
Steps_between_samples = 300 #@param{type:"integer"}

#@markdown * That's it!  Run the cell! or configure these advance options

#@markdown # ________________ ADV SETTINGS _________________
#@markdown These are the default Every Dream 2 settings, changing these without learning what they do will likley waste compute credits
#@markdown please read the doc folder before changing these!

#@markdown * A tip using the sliders, to finely adjust these click them with your mouse then use your keyboard arrows

#@markdown * Using the same seed each time you train allows for more accurate a/b comparison of models, leave at -1 for random
#@markdown * The seed also effects your training samples, if you want the same seed each sample you will need to change it from -1
Training_Seed = -1 #@param{type:"integer"}

#@markdown * use this option to configure a sample_prompts.json
#@markdown * check out /content/EveryDream2trainer/doc/logging.md. for more details
Advance_Samples = False #@param{type:"boolean"}
Sample_File = "sample_prompts.txt"
if Advance_Samples:
  Sample_File = "sample_prompts.json"
#@markdown * Checkpointing Saves Vram to allow larger batch sizes minor slow down on a single batch size but will can allow room for a higher traning resolution (suggested on Colab Free tier, turn off for A100)
Gradient_checkpointing = True #@param{type:"boolean"}
Disable_Xformers = False #@param{type:"boolean"}
#@markdown * Tag shuffling, mainly for booru training. Best to just read this if interested in shufflng tags /content/EveryDream2trainer/doc/SHUFFLING_TAGS.md
shuffle_tags = False #@param{type:"boolean"}
#@markdown * You can turn off the text encoder training (generally not suggested)
Disable_text_Encoder= False #@param{type:"boolean"}
#@markdown * Skip the nth last layer of CLIP.
Clip_skip = 1 #@param {type:"slider", min:0, max:4, step:1}
#@markdown * ratio of captions dropped from training data.
Conditional_DropOut = 0.04 #@param {type:"slider", min:0, max:0.3, step:0.01}
#@markdown * Ratio of images randomly to flip horizontally.
Picture_flip = 0 #@param {type:"slider", min:0, max:0.5, step:0.05}
#@markdown * This can improve contrast in light and dark scenes, Use a ratio between 0-10% for Best results.
zero_frequency_noise = 0.01 #@param {type:"slider", min:0, max:0.12, step:0.01}

#@markdown * Weights and Biases logging token. 
# #@markdown Paste your token here if you have an account so you can use it to track your training progress.  If you don't have an account, you can create one for free at https://wandb.ai/site.  Log will use your project name from above. This is a free online logging utility.
# #@markdown Your key is on this page: https://wandb.ai/settings under "Danger Zone" "API Keys"
wandb_token = '' #@param{type:"string"}


#@markdown use validation with wandb

validatation = False #@param{type:"boolean"}
Hide_Warnings = False #@param {type:"boolean"}

extensions = ['.zip', '.7z', '.rar', '.tgz']
uncompressed_dir = 'Training_Data'

if any(ext in Dataset_Location for ext in extensions):
    # Create the uncompressed directory if it doesn't exist
    if not os.path.exists(uncompressed_dir):
        os.makedirs(uncompressed_dir)
    
    # Extract the compressed file to the uncompressed directory
    shutil.unpack_archive(Dataset_Location, uncompressed_dir)

    # Set the dataset location to the uncompressed directory
    Dataset_Location = uncompressed_dir

# Use the dataset location in the rest of your code
dataset = Dataset_Location

Drive=""
if Save_to_Gdrive:
  Drive = "--logdir /content/drive/MyDrive/everydreamlogs --save_ckpt_dir /content/drive/MyDrive/everydreamlogs/ckpt"

if Max_Epochs==0:
  Max_Epoch=1

if resume:
  model = "findlast"

validate = ""
if validatation:
  validate = "--validation_config validation_default.json"


if Hide_Warnings:
  import warnings
  warnings.filterwarnings("ignore")

wandb_settings = ""
if wandb_token:
  !rm /root/.netrc
  !wandb login $wandb_token
  wandb_settings = "--wandb"

Gradient = ""
if Gradient_checkpointing:
  Gradient = "--gradient_checkpointing "

DX = ""  
if Disable_Xformers:
  DX = "--disable_xformers "

shuffle = ""
if shuffle_tags:
  shuffle = "--shuffle_tags "

Save_optimizer = ""
if Save_optimizer_state:
  Save_optimizer = "--save_optimizer"

textencode = ""
if Disable_text_Encoder:
  textencode = "--disable_textenc_training"

%run train.py --resume_ckpt "$model" \
  $textencode \
  $Gradient \
  $shuffle \
  $Drive \
  $DX \
  $validate \
  $wandb_settings \
  $Save_optimizer \
  --clip_skip $Clip_skip \
  --batch_size $Batch_Size \
  --grad_accum $Gradient_steps \
  --cond_dropout $Conditional_DropOut \
  --data_root "$dataset" \
  --flip_p $Picture_flip \
  --log_step 25 \
  --max_epochs $Max_Epochs \
  --project_name "$Project_Name" \
  --resolution $Resolution \
  --sample_prompts "$Sample_File" \
  --sample_steps $Steps_between_samples \
  --save_every_n_epochs $Save_every_N_epoch \
  --seed $Training_Seed \
  --zero_frequency_noise_ratio $zero_frequency_noise

# Finish the training process
# clear_output()
time.sleep(2)
print("Training is complete, select a model to start training again")
time.sleep(2)

if Disconnect_after_training:
  print("Training is complete, in 30 seconds the instance will reset, you will need to chose a model and you can resume training again")
  time.sleep(40)
  runtime.unassign()

os.kill(os.getpid(), 9)

In [ ]:
#@title Test your Diffusers
import torch
from diffusers import StableDiffusionPipeline, DiffusionPipeline, DPMSolverMultistepScheduler
import torch
#from xformers.ops import MemoryEfficientAttentionFlashAttentionOp

repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

prompt = "High quality photo of an astronaut riding a horse in space"
image = pipe(prompt, num_inference_steps=25).images[0]
image.save("astronaut.png")

#@markdown Path to the diffusers that was trained

#@markdown You can look in the file drawer on the left /content/drive/MyDrive/everydreamlogs and click the three dots to copy the path

#@markdown ex. /content/drive/MyDrive/everydreamlogs/my_project_20230126-023804/ckpts/interrupted-gs86

sd2 = True #@param {type:"boolean"}
diffusers_path="stabilityai/stable-diffusion-2-1-base" #@param{type:"string"}

PROMPT= "a photo of an astronaut on the moon" #@param{type:"string"}
number_of_inference_steps= 35 #@param{type:"integer"}
sample_name = "test" #@param{type:"string"}
n_propmt = "bad, deformed, ugly, bad anotomy"

#pipe.load_textual_inversion("sd-concepts-library/cat-toy")
#pipe.enable_xformers_memory_efficient_attention(attention_op=MemoryEfficientAttentionFlashAttentionOp)



if sd2:
  pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")
else:
  pipe = StableDiffusionPipeline.from_pretrained(diffusers_path, torch_dtype=torch.float16)

pipe = pipe.to("cuda")

image = pipe(PROMPT, num_inference_steps=number_of_inference_steps, negative_prompt=n_propmt).images[0]


image.save(sample_name + ".png")

## Optional NoteBook Features, read all the documentation in /content/EveryDream2trainer/doc before proceeding.

In [ ]:
#@title Remove logs for samples when training (optional) run before training
file_path = "/content/EveryDream2trainer/utils/sample_generator.py"

# Read the file and store the lines
with open(file_path, "r") as file:
    lines = file.readlines()

# The code block to remove
code_to_remove = '''                    with open(f"{self.log_folder}/samples/gs{global_step:05}-{sample_index}-{clean_prompt[:100]}.txt", "w", encoding='utf-8') as f:
                        f.write(str(batch[prompt_idx]))'''

# Join the lines to create a single string
content = ''.join(lines)

# Replace the code block with an empty string
content = content.replace(code_to_remove, '')

# Write the modified content back to the file
with open(file_path, "w") as file:
    file.write(content)

print("The specified code block has been deleted.")

In [ ]:
#@title Alternate startup script
#@markdown * Edit train.json or chain0.json to setup your paramaters

#@markdown * Edit using a chain length of 0 will use train.json

#@markdown * Edit the above cell saving it as chain1.json to make use of chaining

#@markdown * make sure to check each confguration you will need 1 Json per chain length 3 are provided

#@markdown * your locations in the .json can be done in this format /content/drive/MyDrive/   - then the sub folder you wish to use

%cd /content/EveryDream2trainer
Chain_Length=0 #@param{type:"integer"}
l = Chain_Length 
I=0 #repeat counter
if l == None or l == 0:
  train.py --config train.json
while l > 0:
  !python train.py --config chain{I}.json
  l -= 1
  I =+ 1

Need some tools to Manage your large datasets check out https://github.com/victorchall/EveryDream for some usefull tools and captioner